In [2]:
import tensorflow as tf 
import keras as kr 
import numpy as np 

In [5]:
fashion_data = kr.datasets.fashion_mnist

fashion_data

<module 'keras.api.datasets.fashion_mnist' from 'c:\\Users\\jayku\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\keras\\api\\datasets\\fashion_mnist\\__init__.py'>

In [6]:
(train_image, train_labels), (test_image, test_labels) = fashion_data.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 3us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 91s 3us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 3us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 10s 2us/step


In [7]:
test_image = test_image/255.0
train_image = train_image/255.0

In [8]:
train_image = train_image.reshape(len(train_image), 28,28,1)
test_image = test_image.reshape(len(test_image), 28,28,1)


In [20]:
def build_model(hp):
    
    model = kr.Sequential([
        kr.layers.Conv2D(
            filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
            activation='relu',
            input_shape=(28,28,1)
        ),
        kr.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    kr.layers.Flatten(),
    kr.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
     kr.layers.Dense(10, activation='softmax')
    ])
      
    model.compile(optimizer=kr.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
    
    return model

In [21]:
from kerastuner import RandomSearch 
from kerastuner.engine.hyperparameters import HyperParameters

In [22]:
tuner_search=RandomSearch(build_model, objective='val_accuracy',
                          max_trials=5, directory='output', project_name='Mnist Fashion Dataset')

c:\Users\jayku\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuner_search.search(train_image, train_labels, epochs=3, validation_split=0.1)